In [ ]:
# DATA UNDERSTANDING
# --------------------------------------------------------------------------------------

# Collect initial data: Acquire the necessary data and (if necessary) load it into your analysis tool.
# Describe data: Examine the data and document its surface properties like data format, number of records, or field identities.
# Explore data: Dig deeper into the data. Query it, visualize it, and identify relationships among the data.
# Verify data quality: How clean/dirty is the data? Document any quality issues.
# 
# + Visualize the distribution of accidents by various features (e.g., day of the week,
# weather condition, type of road, etc.) using bar plots, histograms, or pie charts.
# + Identify correlations between features and accident severity (e.g., total deaths,
# severe injuries) using correlation matrices or pair plots.

In [ ]:
# CONCLUSIONS
# --------------------------------------------------------------------------------------

# missing cells	187 (< 0.1%)
# mortos has 18698 (94.6%) zeros
# classificacao_acidente has 1072 (5.4%) "Com Vitimas Fatais"
# no interestings correlations
# 
# [x] replace comma by dot: km, latitude and longitude
# [x] filter uf: PR, SC and RS
# [x] specify dtypes

In [46]:
import os

import pandas as pd
from ydata_profiling import ProfileReport

pd.set_option("display.max_columns", None)
YEAR = "2021"

os.makedirs(f"../output/b-data-understanding", exist_ok=True)
os.makedirs(f"../data/b-data-understanding", exist_ok=True)

In [ ]:
# Profilling data grouped by occurrence (datatran2021.csv)
# ======================================================================================

In [7]:
# Read data

file1 = f"../data/raw/{YEAR}-grouped-by-occurrence/datatran2021.csv"

df1 = pd.read_csv(file1, sep=";", decimal=",", encoding="iso-8859-1")
df1.describe(include="all")

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
count,64539.000000,64539,64539,64539,64539,64372.000000,64372.000000,64539,64539,64539,64539,64539,64539,64539,64539,64539,64539,64539.000000,64539.000000,64539.000000,64539.000000,64539.000000,64539.000000,64539.000000,64539.000000,64539.000000,64539.000000,64539,64521,64407
unique,NaN,365,7,1302,27,NaN,NaN,1791,71,17,3,4,3,10,3,10,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,150,369
top,NaN,2021-08-08,sábado,19:00:00,MG,NaN,NaN,BRASILIA,Reação tardia ou ineficiente do condutor,Colisão traseira,Com Vítimas Feridas,Pleno dia,Crescente,Céu Claro,Simples,Reta,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPRF-MG,DEL01-PR,UOP01-DEL01-SC
freq,NaN,281,10869,941,8327,NaN,NaN,981,6907,12312,48186,34785,33898,38983,31805,39054,35812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8305,3032,1035
mean,369745.487194,NaN,NaN,NaN,NaN,210.660163,258.676195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.338508,0.083608,0.839508,0.273710,0.981639,0.160043,1.113218,1.653791,-18.940569,-46.668963,NaN,NaN,NaN
std,21954.249547,NaN,NaN,NaN,NaN,130.673217,224.921727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.866810,0.340734,1.029122,0.612703,1.440071,0.473669,1.151550,0.721653,7.781718,6.186292,NaN,NaN,NaN
min,331693.000000,NaN,NaN,NaN,NaN,10.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-33.689819,-72.119380,NaN,NaN,NaN
25%,351095.500000,NaN,NaN,NaN,NaN,101.000000,78.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,-25.364362,-50.335156,NaN,NaN,NaN
50%,369790.000000,NaN,NaN,NaN,NaN,158.000000,192.650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,2.000000,-20.466319,-47.867100,NaN,NaN,NaN
75%,388100.500000,NaN,NaN,NaN,NaN,324.000000,406.825000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,2.000000,-12.933839,-42.623609,NaN,NaN,NaN


In [8]:
# Basic filtering and data type conversion

dtypes1 = {
    "id": int,
    # Change it to a more specific time-related type if needed (e.g. "datetime64[ns]")
    "data_inversa": "object",
    "dia_semana": "category",
    # Change it to a more specific time-related type if needed
    "horario": "object",
    "uf": "category",
    "br": "category",
    # TODO: category or float64?
    "km": "float64",
    "municipio": "category",
    "causa_acidente": "category",
    "tipo_acidente": "category",
    "classificacao_acidente": "category",
    "fase_dia": "category",
    "sentido_via": "category",
    "condicao_metereologica": "category",
    "tipo_pista": "category",
    "tracado_via": "category",
    "uso_solo": "category",
    "pessoas": int,
    "mortos": int,
    "feridos_leves": int,
    "feridos_graves": int,
    "ilesos": int,
    "ignorados": int,
    "feridos": int,
    "veiculos": int,
    "latitude": "float64",
    "longitude": "float64",
    "regional": "category",
    "delegacia": "category",
    "uop": "category"
}

df1 = df1[df1.uf.isin(["PR", "SC", "RS"])]
df1 = df1.astype(dtypes1)

In [91]:
described1 = df1.describe(include="all")
described1.loc["nan"] = df1.isnull().mean() * 100
described1

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
count,19770.000000,19770,19770,19770,19770,19727.000000,19727.000000,19770,19770,19770,19770,19770,19770,19770,19770,19770,19770,19770.000000,19770.000000,19770.000000,19770.000000,19770.000000,19770.000000,19770.000000,19770.000000,19770.000000,19770.000000,19770,19770,19669
unique,NaN,365,7,903,3,29.000000,NaN,445,69,17,3,4,3,10,3,10,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,29,93
top,NaN,2021-08-08,sábado,19:00:00,SC,101.000000,NaN,CURITIBA,Velocidade Incompatível,Colisão traseira,Com Vítimas Feridas,Pleno dia,Crescente,Céu Claro,Simples,Reta,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPRF-SC,DEL01-PR,UOP01-DEL01-SC
freq,NaN,101,3403,254,7890,4251.000000,NaN,900,2329,3888,14922,10610,10305,10885,9455,11398,10363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7918,3032,1035
mean,369542.644461,NaN,NaN,NaN,NaN,NaN,221.504648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.291047,0.062873,0.837683,0.246232,1.006070,0.138189,1.083915,1.686141,-26.936908,-50.609010,NaN,NaN,NaN
std,21759.173611,NaN,NaN,NaN,NaN,NaN,179.519283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.639392,0.310085,0.944891,0.565921,1.251622,0.399331,1.047696,0.719060,1.969246,1.856358,NaN,NaN,NaN
min,331693.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-33.689819,-57.502806,NaN,NaN,NaN
25%,350965.750000,NaN,NaN,NaN,NaN,NaN,92.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,-28.196260,-51.914951,NaN,NaN,NaN
50%,369583.000000,NaN,NaN,NaN,NaN,NaN,173.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,2.000000,-26.880509,-50.158686,NaN,NaN,NaN
75%,388376.750000,NaN,NaN,NaN,NaN,NaN,313.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,2.000000,-25.502576,-48.956708,NaN,NaN,NaN


In [11]:
df1.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
6,331855,2021-01-01,sexta-feira,15:45:00,PR,277.0,51.3,SAO JOSE DOS PINHAIS,Pista Escorregadia,Saída de leito carroçável,Com Vítimas Feridas,Pleno dia,Decrescente,Garoa/Chuvisco,Dupla,Curva,Não,4,0,1,0,3,0,1,3,-25.595160,-48.907008,SPRF-PR,DEL01-PR,UOP05-DEL01-PR
8,331864,2021-01-01,sexta-feira,17:10:00,SC,470.0,79.1,INDAIAL,Transitar na contramão,Colisão frontal,Com Vítimas Fatais,Pleno dia,Crescente,Nublado,Simples,Reta,Não,9,2,5,1,0,1,6,3,-26.951565,-49.306534,SPRF-SC,DEL04-SC,UOP01-DEL04-SC
10,331910,2021-01-01,sexta-feira,19:50:00,SC,470.0,130.0,LONTRAS,Acessar a via sem observar a presença dos outr...,Colisão transversal,Com Vítimas Feridas,Plena Noite,Decrescente,Nublado,Simples,Não Informado,Sim,4,0,0,2,2,0,2,2,-27.160376,-49.556580,SPRF-SC,DEL04-SC,UOP02-DEL04-SC
11,331920,2021-01-01,sexta-feira,20:40:00,PR,116.0,59.0,CAMPINA GRANDE DO SUL,Entrada inopinada do pedestre,Atropelamento de Pedestre,Com Vítimas Fatais,Plena Noite,Crescente,Chuva,Dupla,Não Informado,Sim,2,1,0,0,1,0,0,1,-25.303357,-48.943789,SPRF-PR,DEL01-PR,UOP02-DEL01-PR
18,331979,2021-01-02,sábado,09:25:00,SC,101.0,112.0,ITAJAI,Trafegar com motocicleta (ou similar) entre as...,Colisão traseira,Com Vítimas Feridas,Pleno dia,Decrescente,Sol,Dupla,Não Informado,Não,4,0,1,1,2,0,2,3,-26.845890,-48.721313,SPRF-SC,DEL04-SC,UOP04-DEL04-SC


In [14]:
# Profile report

profile1 = ProfileReport(df1, title="Profiling Report")
profile1.to_file("../output/b-data-understanding/profile1.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [95]:
# Plotting the data on a map

import plotly.graph_objects as go

fig = go.Figure()

scatter = go.Scattermapbox(
    lat=df1.latitude,
    lon=df1.longitude,
    mode="markers",
    marker=dict(size=5),
    text=df1.causa_acidente,  # Display on hover
)

fig.add_trace(scatter)

fig.update_layout(
    mapbox_style="open-street-map",
    mapbox=dict(center=dict(lat=-28, lon=-52), zoom=4.5),
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)

fig.show()

In [44]:
import plotly.express as px
import numpy as np
import scipy.stats

# To set the z value in a spatial heatmap according to the frequency of data points
# close to each location, you can use Kernel Density Estimation (KDE) to estimate the
# density of points at different locations. KDE is a technique that calculates the
# density of points in a continuous space, which allows you to create a smooth density
# estimation that represents the intensity of data points in the vicinity of each
# location.

data = np.vstack([df1.latitude, df1.longitude])
kde = scipy.stats.gaussian_kde(data)
values = kde(data)

fig = px.density_mapbox(
    lat=df1.latitude,
    lon=df1.longitude,
    z=values,
    radius=4,
)

fig.update_layout(
    mapbox_style="open-street-map",
    mapbox=dict(center=dict(lat=-28, lon=-52), zoom=4.5),
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)

fig.show()

In [47]:
df1.to_csv(f"../data/b-data-understanding/{YEAR}-grouped-by-occurrence.csv", index=False)

In [ ]:
# Profilling data grouped by people (acidentes2021.csv)
# ======================================================================================

In [61]:
# Read data

file2 = f"../data/raw/{YEAR}-grouped-by-people/acidentes2021.csv"

df2 = pd.read_csv(file2, sep=";", decimal=",", encoding="iso-8859-1")
df2.describe(include="all")

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop
count,150925.000000,1.509240e+05,150925,150925,150925,150925,150500.000000,150500.000000,150925,150925,150925,150925,150925,150925,150925,150925,150925,150925,150925.000000,150925,142377,140397.000000,150925,150925,135653.000000,150925,150925.000000,150925.000000,150925.000000,150925.000000,150925.000000,150925.000000,150925,150868,150589
unique,NaN,NaN,365,7,1302,27,NaN,NaN,1791,71,17,3,4,3,10,3,10,2,NaN,25,5638,NaN,6,5,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,27,150,369
top,NaN,NaN,2021-08-08,domingo,19:00:00,MG,NaN,NaN,BRASILIA,Reação tardia ou ineficiente do condutor,Colisão traseira,Com Vítimas Feridas,Pleno dia,Crescente,Céu Claro,Simples,Reta,Não,NaN,Automóvel,VW/GOL 1.0/GOL 1.0/GOL 1.0/GOL 1.0/GOL 1.0/GOL...,NaN,Condutor,Ileso,NaN,Masculino,NaN,NaN,NaN,NaN,NaN,NaN,SPRF-MG,DEL01-PR,UOP01-DEL01-SC
freq,NaN,NaN,733,25638,2174,19058,NaN,NaN,2432,15873,34662,114415,82409,79683,90943,77294,92564,84359,NaN,64113,1728,NaN,106707,63354,NaN,107396,NaN,NaN,NaN,NaN,NaN,NaN,19017,6824,2202
mean,370257.305423,8.262727e+05,NaN,NaN,NaN,NaN,211.213681,261.387833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,668090.466848,NaN,NaN,2010.756092,NaN,NaN,39.606761,NaN,0.419771,0.358993,0.117045,0.035753,-18.775947,-46.675791,NaN,NaN,NaN
std,22338.491324,5.035710e+04,NaN,NaN,NaN,NaN,130.729808,226.563315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42070.510589,NaN,NaN,7.739965,NaN,NaN,46.331935,NaN,0.493523,0.479707,0.321475,0.185674,7.826366,6.214617,NaN,NaN,NaN
min,331693.000000,7.382040e+05,NaN,NaN,NaN,NaN,10.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,595095.000000,NaN,NaN,1900.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,-33.689819,-72.119380,NaN,NaN,NaN
25%,351360.000000,7.833618e+05,NaN,NaN,NaN,NaN,101.000000,79.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,632034.000000,NaN,NaN,2007.000000,NaN,NaN,27.000000,NaN,0.000000,0.000000,0.000000,0.000000,-25.271278,-50.365760,NaN,NaN,NaN
50%,370459.000000,8.263025e+05,NaN,NaN,NaN,NaN,158.000000,195.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,668743.000000,NaN,NaN,2012.000000,NaN,NaN,37.000000,NaN,0.000000,0.000000,0.000000,0.000000,-20.353744,-47.775077,NaN,NaN,NaN
75%,388722.000000,8.685342e+05,NaN,NaN,NaN,NaN,324.000000,410.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,703526.000000,NaN,NaN,2017.000000,NaN,NaN,48.000000,NaN,1.000000,1.000000,0.000000,0.000000,-12.725957,-42.600677,NaN,NaN,NaN


In [76]:
# Basic filtering and data type conversion

# df2 = df2.drop(columns=["pesid", "id_veiculo"])

dtypes2 = {
    "id": int,
    "pesid": int,
    # Change it to a more specific time-related type if needed (e.g. "datetime64[ns]")
    "data_inversa": "object",
    "dia_semana": "category",
    # Change it to a more specific time-related type if needed
    "horario": "category",
    "uf": "category",
    "br": "category",
    # TODO: category or float64?
    "km": "float64",
    "municipio": "category",
    "causa_acidente": "category",
    "tipo_acidente": "category",
    "classificacao_acidente": "category",
    "fase_dia": "category",
    "sentido_via": "category",
    "condicao_metereologica": "category",
    "tipo_pista": "category",
    "tracado_via": "category",
    "uso_solo": "category",
    "id_veiculo": int,
    "tipo_veiculo": "category",
    "marca": "category",
    # IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer
    # "Int64": Nullable integer data type
    "ano_fabricacao_veiculo": "Int64",
    "tipo_envolvido": "category",
    "estado_fisico": "category",
    # IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer
    # "Int64": Nullable integer data type
    "idade": "Int64",
    "sexo": "category",
    "ilesos": int,
    "feridos_leves": int,
    "feridos_graves": int,
    "mortos": int,
    "latitude": "float64",
    "longitude": "float64",
    "regional": "category",
    "delegacia": "category",
    "uop": "category"
}

df2 = df2[df2.uf.isin(["PR", "SC", "RS"])]
df2 = df2.astype(dtypes2)

In [88]:
described2 = df2.describe(include="all")
described2.loc["nan"] = df2.isnull().mean() * 100
described2

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop
count,45294.000000,45294.000000,45294,45294,45294,45294,45189.000000,45189.000000,45294,45294,45294,45294,45294,45294,45294,45294,45294,45294,45294.000000,45294,43363,42985.0,45294,45294,41434.0,45294,45294.000000,45294.000000,45294.000000,45294.000000,45294.000000,45294.000000,45294,45294,45050
unique,NaN,NaN,365,7,903,3,29.000000,NaN,445,69,17,3,4,3,10,3,10,2,NaN,23,4051,<NA>,5,5,<NA>,4,NaN,NaN,NaN,NaN,NaN,NaN,3,29,93
top,NaN,NaN,2021-08-08,sábado,19:00:00,SC,101.000000,NaN,CURITIBA,Velocidade Incompatível,Colisão traseira,Com Vítimas Feridas,Pleno dia,Crescente,Céu Claro,Simples,Reta,Não,NaN,Automóvel,VW/GOL 1.0/GOL 1.0/GOL 1.0/GOL 1.0/GOL 1.0/GOL...,<NA>,Condutor,Ileso,<NA>,Masculino,NaN,NaN,NaN,NaN,NaN,NaN,SPRF-SC,DEL01-PR,UOP01-DEL01-SC
freq,NaN,NaN,254,7949,629,17700,9279.000000,NaN,1967,4762,10661,35251,25015,23909,25188,22331,26154,23837,NaN,21667,580,<NA>,33324,19890,<NA>,31936,NaN,NaN,NaN,NaN,NaN,NaN,17761,6824,2202
mean,369779.087738,825204.775467,NaN,NaN,NaN,NaN,NaN,223.602494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,667259.903144,NaN,NaN,2009.707945,NaN,NaN,38.957692,NaN,0.439131,0.365633,0.107476,0.027443,-26.935300,-50.643771,NaN,NaN,NaN
std,21912.938871,49683.469822,NaN,NaN,NaN,NaN,NaN,180.635497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41577.640559,NaN,NaN,8.242937,NaN,NaN,45.466436,NaN,0.496287,0.481613,0.309720,0.163372,1.972519,1.853267,NaN,NaN,NaN
min,331693.000000,738216.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,595103.000000,NaN,NaN,1900.0,NaN,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,-33.689819,-57.502806,NaN,NaN,NaN
25%,351063.000000,782620.250000,NaN,NaN,NaN,NaN,NaN,92.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,631509.250000,NaN,NaN,2006.0,NaN,NaN,26.0,NaN,0.000000,0.000000,0.000000,0.000000,-28.237665,-51.937500,NaN,NaN,NaN
50%,369945.000000,824888.500000,NaN,NaN,NaN,NaN,NaN,174.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,667671.000000,NaN,NaN,2011.0,NaN,NaN,36.0,NaN,0.000000,0.000000,0.000000,0.000000,-26.879056,-50.228897,NaN,NaN,NaN
75%,388620.000000,868126.750000,NaN,NaN,NaN,NaN,NaN,321.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,703295.000000,NaN,NaN,2016.0,NaN,NaN,48.0,NaN,1.000000,1.000000,0.000000,0.000000,-25.498782,-48.982243,NaN,NaN,NaN


In [78]:
df2.head()

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop
7,331855,738445,2021-01-01,sexta-feira,15:45:00,PR,277.0,51.3,SAO JOSE DOS PINHAIS,Pista Escorregadia,Saída de leito carroçável,Com Vítimas Feridas,Pleno dia,Decrescente,Garoa/Chuvisco,Dupla,Curva,Não,595297,Caminhão,VW/8.160 DRC 4X2/8.160 DRC 4X2/8.160 DRC 4X2/8...,2016,Condutor,Ileso,39,Masculino,1,0,0,0,-25.595160,-48.907008,SPRF-PR,DEL01-PR,UOP05-DEL01-PR
9,331864,738673,2021-01-01,sexta-feira,17:10:00,SC,470.0,79.1,INDAIAL,Transitar na contramão,Colisão frontal,Com Vítimas Fatais,Pleno dia,Crescente,Nublado,Simples,Reta,Não,595462,Caminhão,FORD/CARGO 2422 T/CARGO 2422 T/CARGO 2422 T/CA...,2005,Passageiro,Lesões Leves,27,Feminino,0,1,0,0,-26.951565,-49.306534,SPRF-SC,DEL04-SC,UOP01-DEL04-SC
10,331864,738672,2021-01-01,sexta-feira,17:10:00,SC,470.0,79.1,INDAIAL,Transitar na contramão,Colisão frontal,Com Vítimas Fatais,Pleno dia,Crescente,Nublado,Simples,Reta,Não,595462,Caminhão,FORD/CARGO 2422 T/CARGO 2422 T/CARGO 2422 T/CA...,2005,Condutor,Lesões Leves,24,Masculino,0,1,0,0,-26.951565,-49.306534,SPRF-SC,DEL04-SC,UOP01-DEL04-SC
11,331864,741989,2021-01-01,sexta-feira,17:10:00,SC,470.0,79.1,INDAIAL,Transitar na contramão,Colisão frontal,Com Vítimas Fatais,Pleno dia,Crescente,Nublado,Simples,Reta,Não,595462,Caminhão,FORD/CARGO 2422 T/CARGO 2422 T/CARGO 2422 T/CA...,2005,Testemunha,Não Informado,<NA>,Não Informado,0,0,0,0,-26.951565,-49.306534,SPRF-SC,DEL04-SC,UOP01-DEL04-SC
13,331910,738556,2021-01-01,sexta-feira,19:50:00,SC,470.0,130.0,LONTRAS,Acessar a via sem observar a presença dos outr...,Colisão transversal,Com Vítimas Feridas,Plena Noite,Decrescente,Nublado,Simples,Não Informado,Sim,595375,Caminhão-trator,SCANIA/P 360 A4X2/P 360 A4X2/P 360 A4X2/P 360 ...,2018,Condutor,Ileso,37,Masculino,1,0,0,0,-27.160376,-49.556580,SPRF-SC,DEL04-SC,UOP02-DEL04-SC


In [79]:
df2.to_csv(f"../data/b-data-understanding/{YEAR}-grouped-by-people.csv", index=False)

In [80]:
# Profile report

profile2 = ProfileReport(df2, title="Profiling Report")
profile2.to_file("../output/b-data-understanding/profile2.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Profilling data grouped by people all (acidentes2021_todas_causas_tipos.csv)
# ======================================================================================

In [82]:
# Read data

file3 = f"../data/raw/{YEAR}-grouped-by-people-all/acidentes2021_todas_causas_tipos.csv"

df3 = pd.read_csv(file3, sep=";", decimal=",", encoding="iso-8859-1")
df3.describe(include="all")

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_principal,causa_acidente,ordem_tipo_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop
count,449018.000000,4.091470e+05,449018,449018,449018,449018,447870.000000,447870.000000,449018,449018,449018,449018.000000,449018,449018,449018,449018,449018,449018,449018,449018,449018.000000,449018,428080,422909.000000,449018,449018,359543.000000,449018,449018.000000,449018.000000,449018.000000,449018.000000,449018.000000,449018.000000,449018,448872,447950
unique,NaN,NaN,365,7,1302,27,NaN,NaN,1791,2,72,NaN,17,3,4,3,10,3,10,2,NaN,25,6555,NaN,6,5,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,27,150,369
top,NaN,NaN,2021-07-07,domingo,17:30:00,MG,NaN,NaN,BRASILIA,Sim,Velocidade Incompatível,NaN,Colisão traseira,Com Vítimas Feridas,Pleno dia,Crescente,Céu Claro,Simples,Reta,Não,NaN,Automóvel,SR/RANDON SR CA/RANDON SR CA/RANDON SR CA/RAND...,NaN,Condutor,Ileso,NaN,Masculino,NaN,NaN,NaN,NaN,NaN,NaN,SPRF-MG,DEL01-PR,UOP01-DEL01-SP
freq,NaN,NaN,3323,79924,6702,60684,NaN,NaN,6502,279255,55869,NaN,69234,321699,245374,235500,263156,244889,258215,271947,NaN,165554,6775,NaN,268165,158094,NaN,280628,NaN,NaN,NaN,NaN,NaN,NaN,60476,16233,6612
mean,370948.730173,8.278644e+05,NaN,NaN,NaN,NaN,214.081439,279.217411,NaN,NaN,NaN,1.714412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,669342.901612,NaN,NaN,2010.889033,NaN,NaN,39.078288,NaN,0.352088,0.308181,0.119307,0.046593,-17.856424,-47.141010,NaN,NaN,NaN
std,22768.496844,5.101182e+04,NaN,NaN,NaN,NaN,129.088957,237.307300,NaN,NaN,NaN,1.124472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42481.100951,NaN,NaN,7.657329,NaN,NaN,42.743596,NaN,0.477622,0.461742,0.324150,0.210765,7.895926,6.406743,NaN,NaN,NaN
min,331693.000000,7.382040e+05,NaN,NaN,NaN,NaN,10.000000,0.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,595095.000000,NaN,NaN,1900.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,-33.689819,-72.119380,NaN,NaN,NaN
25%,352086.000000,7.849650e+05,NaN,NaN,NaN,NaN,101.000000,83.500000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,633504.000000,NaN,NaN,2008.000000,NaN,NaN,27.000000,NaN,0.000000,0.000000,0.000000,0.000000,-23.618093,-51.115413,NaN,NaN,NaN
50%,370970.000000,8.277810e+05,NaN,NaN,NaN,NaN,163.000000,217.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,669808.500000,NaN,NaN,2012.000000,NaN,NaN,37.000000,NaN,0.000000,0.000000,0.000000,0.000000,-19.776540,-47.817140,NaN,NaN,NaN
75%,389188.000000,8.698450e+05,NaN,NaN,NaN,NaN,343.000000,433.100000,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,704382.000000,NaN,NaN,2017.000000,NaN,NaN,48.000000,NaN,1.000000,1.000000,0.000000,0.000000,-11.435201,-43.023224,NaN,NaN,NaN


In [84]:
# Basic filtering and data type conversion

dtypes3 = {
    "id": int,
    # IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer
    # "Int64": Nullable integer data type
    "pesid": "Int64",
    # Change it to a more specific time-related type if needed (e.g. "datetime64[ns]")
    "data_inversa": "category",
    "dia_semana": "category",
    # Change it to a more specific time-related type if needed
    "horario": "category",
    "uf": "category",
    "br": "category",
    # TODO: category or float64?
    "km": "float64",
    "municipio": "category",
    "causa_principal": "category",
    "causa_acidente": "category",
    "ordem_tipo_acidente": int,
    "tipo_acidente": "category",
    "classificacao_acidente": "category",
    "fase_dia": "category",
    "sentido_via": "category",
    "condicao_metereologica": "category",
    "tipo_pista": "category",
    "tracado_via": "category",
    "uso_solo": "category",
    "id_veiculo": int,
    "tipo_veiculo": "category",
    "marca": "category",
    # IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer
    # "Int64": Nullable integer data type
    "ano_fabricacao_veiculo": "Int64",
    "tipo_envolvido": "category",
    "estado_fisico": "category",
    # IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer
    # "Int64": Nullable integer data type
    "idade": "Int64",
    "sexo": "category",
    "ilesos": int,
    "feridos_leves": int,
    "feridos_graves": int,
    "mortos": int,
    "latitude": "float64",
    "longitude": "float64",
    "regional": "category",
    "delegacia": "category",
    "uop": "category"
}

df3 = df3[df3.uf.isin(["PR", "SC", "RS"])]
df3 = df3.astype(dtypes3)

In [87]:
described3 = df3.describe(include="all")
described3.loc["nan"] = df3.isnull().mean() * 100
described3

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_principal,causa_acidente,ordem_tipo_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop
count,109799.000000,101652.0,109799,109799,109799,109799,109571.000000,109571.000000,109799,109799,109799,109799.000000,109799,109799,109799,109799,109799,109799,109799,109799,109799.000000,109799,105818,105004.0,109799,109799,90572.0,109799,109799.000000,109799.000000,109799.000000,109799.000000,109799.000000,109799.000000,109799,109799,108997
unique,NaN,<NA>,365,7,903,3,29.000000,NaN,445,2,71,NaN,17,3,4,3,10,3,10,2,NaN,23,4570,<NA>,6,5,<NA>,4,NaN,NaN,NaN,NaN,NaN,NaN,3,29,93
top,NaN,<NA>,2021-06-02,sábado,17:30:00,PR,101.000000,NaN,CURITIBA,Sim,Velocidade Incompatível,NaN,Colisão traseira,Com Vítimas Feridas,Pleno dia,Crescente,Céu Claro,Simples,Reta,Não,NaN,Automóvel,VW/GOL 1.0/GOL 1.0/GOL 1.0/GOL 1.0/GOL 1.0/GOL...,<NA>,Condutor,Ileso,<NA>,Masculino,NaN,NaN,NaN,NaN,NaN,NaN,SPRF-PR,DEL01-PR,UOP01-DEL01-SC
freq,NaN,<NA>,1109,19637,1972,42176,18466.000000,NaN,4235,77979,15157,NaN,18239,82290,59838,58013,59422,56383,58658,61621,NaN,47494,1226,<NA>,69728,41426,<NA>,69068,NaN,NaN,NaN,NaN,NaN,NaN,42079,16233,4263
mean,370187.840363,826168.967831,NaN,NaN,NaN,NaN,NaN,230.553164,NaN,NaN,NaN,1.593739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,668074.641791,NaN,NaN,2009.665013,NaN,NaN,38.569017,NaN,0.377289,0.322061,0.107606,0.037960,-26.968303,-50.804915,NaN,NaN,NaN
std,21824.596364,49565.051969,NaN,NaN,NaN,NaN,NaN,181.026074,NaN,NaN,NaN,0.954048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41388.098931,NaN,NaN,8.241989,NaN,NaN,45.819972,NaN,0.484710,0.467268,0.309883,0.191101,2.032691,1.850636,NaN,NaN,NaN
min,331693.000000,738216.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,595103.000000,NaN,NaN,1900.0,NaN,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,-33.689819,-57.502806,NaN,NaN,NaN
25%,351768.000000,784240.75,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,632905.000000,NaN,NaN,2006.0,NaN,NaN,26.0,NaN,0.000000,0.000000,0.000000,0.000000,-28.455675,-52.046432,NaN,NaN,NaN
50%,370439.000000,826280.0,NaN,NaN,NaN,NaN,NaN,184.600000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,668645.000000,NaN,NaN,2011.0,NaN,NaN,36.0,NaN,0.000000,0.000000,0.000000,0.000000,-26.881372,-50.651211,NaN,NaN,NaN
75%,388768.000000,868731.0,NaN,NaN,NaN,NaN,NaN,330.800000,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,703668.500000,NaN,NaN,2016.0,NaN,NaN,47.0,NaN,1.000000,1.000000,0.000000,0.000000,-25.478651,-49.100332,NaN,NaN,NaN


In [89]:
df3.head()

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_principal,causa_acidente,ordem_tipo_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop
0,331693,739775,2021-01-01,sexta-feira,00:02:00,SC,101.0,314.9,LAGUNA,Sim,Ingestão de álcool pelo condutor,3,Capotamento,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Dupla,Reta,Não,595104,Automóvel,FIAT/UNO VIVACE 1.0/UNO VIVACE 1.0/UNO VIVACE ...,2010,Condutor,Ileso,35,Masculino,1,0,0,0,-28.430294,-48.849251,SPRF-SC,DEL02-SC,UOP01-DEL02-SC
1,331693,739774,2021-01-01,sexta-feira,00:02:00,SC,101.0,314.9,LAGUNA,Sim,Ingestão de álcool pelo condutor,3,Capotamento,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Dupla,Reta,Não,595103,Automóvel,I/RENAULT CLIO CAM 10H3P/RENAULT CLIO CAM 10H3...,2010,Passageiro,Lesões Leves,22,Feminino,0,1,0,0,-28.430294,-48.849251,SPRF-SC,DEL02-SC,UOP01-DEL02-SC
2,331693,738219,2021-01-01,sexta-feira,00:02:00,SC,101.0,314.9,LAGUNA,Sim,Ingestão de álcool pelo condutor,3,Capotamento,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Dupla,Reta,Não,595103,Automóvel,I/RENAULT CLIO CAM 10H3P/RENAULT CLIO CAM 10H3...,2010,Condutor,Ileso,27,Masculino,1,0,0,0,-28.430294,-48.849251,SPRF-SC,DEL02-SC,UOP01-DEL02-SC
3,331693,739775,2021-01-01,sexta-feira,00:02:00,SC,101.0,314.9,LAGUNA,Sim,Ingestão de álcool pelo condutor,2,Colisão com objeto,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Dupla,Reta,Não,595104,Automóvel,FIAT/UNO VIVACE 1.0/UNO VIVACE 1.0/UNO VIVACE ...,2010,Condutor,Ileso,35,Masculino,1,0,0,0,-28.430294,-48.849251,SPRF-SC,DEL02-SC,UOP01-DEL02-SC
4,331693,739774,2021-01-01,sexta-feira,00:02:00,SC,101.0,314.9,LAGUNA,Sim,Ingestão de álcool pelo condutor,2,Colisão com objeto,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Dupla,Reta,Não,595103,Automóvel,I/RENAULT CLIO CAM 10H3P/RENAULT CLIO CAM 10H3...,2010,Passageiro,Lesões Leves,22,Feminino,0,1,0,0,-28.430294,-48.849251,SPRF-SC,DEL02-SC,UOP01-DEL02-SC


In [90]:
# Profile report

profile3 = ProfileReport(df3, title="Profiling Report")
profile3.to_file("../output/b-data-understanding/profile3.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [93]:
df3.to_csv(f"../data/b-data-understanding/{YEAR}-grouped-by-people-all.csv", index=False)